In [1]:
import os
import sys

In [2]:
# Update env and pkg paths.

sys.path.extend(['/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages'])
print(sys.path)

['/data/xzuo/Projects/dissertation/dataset_URL_parser/pmc_data_linker', '/usr/local/anaconda3/lib/python39.zip', '/usr/local/anaconda3/lib/python3.9', '/usr/local/anaconda3/lib/python3.9/lib-dynload', '', '/home/xzuo/.local/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/usr/local/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/xzuo/.ipython', '/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages']


## Load citation links

In [3]:
import re
from lxml import etree

In [4]:
# Get pmid from LitCovid.

id_list_path = "/data/xzuo/Projects/dissertation/dataset_URL_parser/paper_oriented/paper_litcovid_pmid/litcovid_16052022.tsv"
csv_directory_path = "/data/xzuo/Projects/dissertation/dataset_URL_parser/paper_oriented/litcovid_pmcid_lists"
download_directory_path ="/data/xzuo/Projects/dissertation/dataset_URL_parser/paper_oriented/litcovid_pmc_fulltext/xml"

current_filelist = os.listdir(download_directory_path)

In [5]:
# Remove inaccessible files.

inaccessible_articles = []

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    try:
        pmc_tree = etree.parse(pmc_xml_path)
    except:
        # print("Remove file %s." % xml_name)
        inaccessible_articles.append(xml_name)
        current_filelist.remove(xml_name)

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    pmc_tree = etree.parse(pmc_xml_path)
    meta = pmc_tree.find(".//article-meta")
    if meta is None:
        inaccessible_articles.append(xml_name)
        current_filelist.remove(xml_name)
#         print("File %s is not accessible." % xml_name)
    else:
        continue

print("There are %d articles not accessible." % len(inaccessible_articles))

for i, xml_name in enumerate(inaccessible_articles):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    if os.path.exists(pmc_xml_path):
        os.remove(pmc_xml_path)
    else:
        print("Error: File %s does not exist." % xml_name)

There are 0 articles not accessible.


In [7]:
from pubmed_oa_parser import parse_pubmed_xml
from pmc_data_parser import parse_pubmed_references, parse_das_sec, parse_in_text_url, PMCDataElement

In [8]:
articles = []

for i, pmc_file in enumerate(current_filelist):
    pmc_file_path = os.path.join(download_directory_path, pmc_file)
    pmc_meta = parse_pubmed_xml(pmc_file_path)
    refs = parse_pubmed_references(pmc_file_path)
    das_sec = parse_das_sec(pmc_file_path)
    in_text_urls = parse_in_text_url(pmc_file_path)
    pmc_article = PMCDataElement(pmc_meta['pmid'], pmc_meta['pmc'], pmc_meta['full_title'], pmc_meta['abstract'], das_sec, refs, in_text_urls)
    # print(pmc.pmid, pmc.das, pmc.ref, pmc.in_text_urls)
    articles.append(pmc_article)
    if i % 10000 == 0:
        print("Processed %d files." % (i+1))
        # break

Processed 1 files.
Processed 10001 files.
Processed 20001 files.
Processed 30001 files.
Processed 40001 files.
Processed 50001 files.
Processed 60001 files.
Processed 70001 files.
Processed 80001 files.
Processed 90001 files.
Processed 100001 files.


## Search datasets

In [12]:
# name_patterns = re.compile(r'(N3C)|(National COVID Cohort Collaborative)|(GenBank)|\
#                      (GISAID)|(Nextstrain)|(OpenICPSR)|(CORD-19)|(LitCovid)|(NCBI Virus)')

# url_patterns = re.compile(r'(covid.cdc.gov)|(covid19dataportal)|(worldometers)|(CSSEGISandData)')

# name_patterns = re.compile(r'(EMBL-EBI)|(GEO)|(Gene Expression Omnibus)|(Reactome)|(Protein Data Bank)')
# url_patterns = re.compile(r'(ourworldindata)|(data.europa.eu)|(nytimes)')

name_patterns = re.compile(r'((?:machine|deep?)\s*(?:learning))|(Cancer Imaging Archive)|(TCIA)|(Medical Imaging and Data Resource Center)|(MIDRC)')
url_patterns = re.compile(r'(github)|(kaggle)|()')

In [13]:
match_count = 0

for i, article in enumerate(articles):
    name_first_match = ""
    url_first_match = ""
    
    if article.refs is not None: 
        ref_text = " ".join([ref['journal'] for ref in article.refs])
    else:
        ref_text = ""
    text = " ".join([article.title, article.abstract, article.das['das_statement'], ref_text])
    if name_patterns.search(text):
        name_first_match = name_patterns.search(text).group(0)
        # print(article.pmcid, first_match)
    
    if article.refs is not None:
        ref_urls = [ref['url'] for ref in article.refs if ref['url'] != ""]
    else:
        ref_urls = []
    das_urls = article.das['das_links'] 
    urls = ref_urls + das_urls
    if len(urls) > 0:
        # print(article.pmcid, urls)
        for url in urls:
            if url_patterns.search(str(url)):
                url_first_match = url_patterns.search(str(url)).group(0)
                # print(article.pmcid, first_match)
                break
    else:
        continue
    if name_first_match != "" or url_first_match != "":
        print("%s\t%s\t%s" % (article.pmcid, name_first_match, url_first_match))
        match_count += 1
print("Searched %d files. Found %d matches." % (i+1, match_count))

8675556	deep learning	
8848635	machine learning	
8275616	deep learning	
7941334	machine learning	
8898749	machine learning	
8043493	machine learning	
8783177	machine learning	
7980745		github
8545167	deep learning	
8446042	deep learning	
8043496	deep learning	
9035978	machine learning	
7829098	deep learning	
7945584	machine learning	
8865947		github
8722646	deep learning	
8079380	machine learning	
8852237	machine learning	
7581467	deep learning	
8545306	machine learning	
8260732	machine learning	
8329284	machine learning	
8043420	deep learning	
7723858	deep learning	
8290951	machine learning	
8993675	machine learning	
8843052	deep learning	
7869774	deep learning	
8545278	deep learning	
8857913	deep learning	
7775454	machine learning	
8898191	machine learning	
8767777	machine learning	
7571315	machine learning	
8214048	machine learning	
7436068	machine learning	
8367981	deep learning	
8200785	machine learning	
7673985	machine learning	
8545310	machine learning	
8065335	machine learning	